# Web Scrapping

In [11]:
###### ใช้ response ไม่โดน clouflare จับ ######
### Main + detail ###

# div class="listing-card listing-id-3819320 listing-card-sale featured-item ranked" >> box of content
#   a class="nav-link" >> Link , Tsitle
#   p class="listing-location ellipsis" >> address
#   span class="price" >> price
#   span class="bed" >> bedroom
#   span class="bath" >> bathroom
#   li class="listing-floorarea pull-left" >> room size
#   li class="listing-floorarea pull-left" >> room size
#   ul data-automation-id="listing-features-walk" >> near bts,mrt


from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np

list_df = []
list_link = []
real_ip = '52.77.133.102'

for page in range(0,1):#0 500

    URL = 'https://' + real_ip +'/%E0%B8%A3%E0%B8%A7%E0%B8%A1%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%81%E0%B8%B2%E0%B8%A8%E0%B8%82%E0%B8%B2%E0%B8%A2/'  + str(page) + '?property_type=N&property_type_code%5B0%5D=CONDO'

    headers = {'host': 'www.ddproperty.com','User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}
    response = requests.get(URL, headers=headers, verify=False)
    response.encoding = 'utf-8'

    htmlDoc = BeautifulSoup(response.text, 'html.parser')


    #all_box = htmlDoc.find_all('div' , attrs={'class' : re.compile(r'^(listing-card).*(ranked)$')}) ## filter only class starting with "listing-card" and ending with "ranked"
    all_box = htmlDoc.find_all('div' , attrs={'class' : re.compile(r'^(listing-card listing-id-)\d+ listing-card-sale.*')}) 
    

    l_data = []


    # Loop each box
    for box in all_box:
        d_data = dict()


        #### scrapping main information

        # get title and link
        a_tag = box.find('a', attrs={'class' : 'nav-link'})

        if a_tag["href"] not in list_link:

            d_data['title'] = a_tag['title']
            d_data['link'] = a_tag["href"]
            list_link.append(a_tag["href"])

            # get address
            p_tag = box.find('p', attrs={'class' : 'listing-location ellipsis'})

            address = p_tag.text.split(sep=',')
            if len(address) == 4: # มีแขวง, เขต, จังหวัด ครบ
                d_data['address'] = p_tag.text
                d_data['province'] = address[-1]
                d_data['district'] = address[-2]
                d_data['city'] = address[-3]
            else: # มีแขวง, เขต, จังหวัด ไม่ครบ ไม่เอามา fit model
                continue

            

            
            # get price
            span_price = box.find('span', attrs={'class' : 'price'})
            
            if span_price:
                price = span_price.text.replace(',','')
                if price.isnumeric():
                    d_data['price'] = int(price)
                else:
                    continue
            else:
                continue

            # get bedroom
            span_bed = box.find('span', attrs={'class' : re.compile(r'^(bed|studio)')})

            if span_bed:
                if span_bed['class'][0] == 'studio':
                    d_data['bedroom'] = 1
                else:
                    d_data['bedroom'] = int(span_bed.text)
            else:
                d_data['bedroom'] = np.nan

            #get bathroom
            span_bath = box.find('span', attrs={'class' : 'bath'})
            
            if span_bath:
                d_data['bathroom'] = int(span_bath.text)
            else: # if no bath tag found. default 1 bathroom
                d_data['bathroom'] = 1


            # get room size and price per meter^2
            li_tag = box.find_all('li', attrs={'class' : 'listing-floorarea pull-left'})

            count = 0
            for li in li_tag:
                count += 1
                if count == 1:
                    d_data['size'] = ''.join(re.findall(r'\d+',li.text))
                else:
                    d_data['price_per_meter2'] = ''.join(re.findall(r'\d+',li.text.replace(',','')))


            # get bts,mrt info
            ul_station = box.find('ul', attrs={'data-automation-id' : 'listing-features-walk'})
            if ul_station:
                d_data['nearing_station'] = str.strip(ul_station.text)
                d_data['distance_station'] = ''.join(re.findall(r'(?<=\()\d+', str.strip(ul_station.text)))

            else:
                d_data['nearing_station'] = np.nan
                d_data['distance_station'] = np.nan
    ####        

            #### scrapping detail information
            URL2 = d_data['link'].replace('www.ddproperty.com', real_ip)
            response_detail = requests.get(URL2, headers=headers, verify=False)
            response_detail.encoding = 'utf-8'

            htmlDoc2 = BeautifulSoup(response_detail.text, 'html.parser')

            all_tr = htmlDoc2.find_all('tr',attrs={'class' : re.compile(r'(property-attr).*')})
        


            count = 1
            for tr in all_tr:

                
                if count == 4:
                    # เจ้าของโครงการ
                    prj_t = tr.find('td', attrs={'class' : 'value-block'})
                    d_data['prj_owner'] = prj_t.text
                elif count == 7:
                    # Funiture
                    fur_t = tr.find('td', attrs={'class' : 'value-block'})
                    d_data['furniture'] = fur_t.text
                elif count == 8:
                    # ปีที่สร้างเสร็จ
                    fin_t = tr.find('td', attrs={'class' : 'value-block'})
                    d_data['finish_year'] = fin_t.text
                elif count == 9:
                    # ชั้น
                    floor_t = tr.find('td', attrs={'class' : 'value-block'})
                    d_data['floor'] = floor_t.text
                count += 1
            
            condo_t = htmlDoc2.find('div', attrs={'id' : 'condo-profile'})

            # ชื่อ Condo
            if condo_t:
                condo = condo_t.find('span', attrs={'class' : 'condo-profile-box__title-name'})
                if condo:
                    d_data['condo'] = condo.text
                else:
                    d_data['condo'] = np.nan
            else:
                d_data['condo'] = np.nan

    ####
            l_data.append(d_data)

        # duplicating box. so skip to next box
        else:
            continue

    df = pd.DataFrame(l_data)
    list_df.append(df)


all_df = pd.concat(list_df, ignore_index=True)
all_df.info()



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '52.77.133.102'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame

In [253]:
# export to csv
all_df.to_csv('main_detail1.csv', encoding='utf-8', index = False)

NameError: name 'all_df' is not defined

In [ ]:
# Disable warning
import urllib3
urllib3.disable_warnings()

# Re enable warning
# import warnings
# warnings.resetwarnings()

# Import Data

In [1]:
import pandas as pd
import numpy as np

df_main = pd.read_csv('all_main_detail.csv')

In [2]:
# scope เฉพาะ condo ใน กรุงเทพ
df_main['province'] = df_main['province'].apply(lambda x: x.strip())
df_main = df_main[df_main['province'] == 'กรุงเทพ']

# drop column ที่ไม่ใช้ออก
df_main.drop_duplicates(subset=['link'], inplace=True, ignore_index=True)
df_main.drop(['title', 'address','nearing_station', 'province', 'prj_owner', 'condo', 'price_per_meter2'],axis=1 , inplace=True)

# ลบ space ออก เช่น ' district'
df_main['district'] = df_main['district'].apply(lambda x: x.strip())
df_main['city'] = df_main['city'].apply(lambda x: x.strip())



df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78396 entries, 0 to 78395
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   link              78396 non-null  object 
 1   district          78396 non-null  object 
 2   city              78396 non-null  object 
 3   price             78396 non-null  int64  
 4   bedroom           77549 non-null  float64
 5   bathroom          78396 non-null  int64  
 6   size              78290 non-null  float64
 7   distance_station  62423 non-null  float64
 8   furniture         78395 non-null  object 
 9   finish_year       78395 non-null  object 
 10  floor             78395 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 6.6+ MB


# Data cleansing

In [3]:
# เจอจำนวน ห้องนอน ผิดปกติ เลยเอาข้อมูลที่ถูกมาใส่แทน
df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/6410-189-ขาย-คอนโด-รัชดา-พระราม-9-เพชรบุรีตัดใหม่-thru-thonglor-2ห้องน-ขาย-9624155', 'bedroom'] = 2
df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/6411-652-ขาย-คอนโด-บางนา-แบริ่ง-btsบางนา-supalai-city-resort-bearing-s-ขาย-9764540', 'bedroom'] = 2
df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/ให้เช่า-city-living-ratchada-ซิตี้-ลีฟวิ่ง-รัชดา-ใกล้-mrt-ห้วยขวาง-pp9re-10051-ขาย-9943062', 'bedroom'] = 1
df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/ขายด่วน-คอนโด-ราคาถูกสุดๆ-a-space-เกษตร-35-ตรม-ชั้น-2-เพียง-1-5-ล้าน-ขาย-8818860', 'bedroom'] = 1


# drop row ที่ bedroom อาจใส่ผิด หรือ เป็นการปล่อยเช่า
df_main.drop(index = df_main[(df_main['bedroom'] > 2) & (df_main['price'] < 1000000)].index, inplace=True)
# df_main[(df_main['bedroom'] > 2) & (df_main['price'] < 1000000)]

df_main['bedroom'] = pd.to_numeric(df_main['bedroom'], errors='coerce')

df_main['bedroom'].value_counts().sort_index(ascending=False)




10.0        3
8.0         1
7.0         9
6.0        60
5.0       212
4.0      1030
3.0      5285
2.0     20979
1.0     49945
Name: bedroom, dtype: int64

In [4]:
# เจอจำนวน ห้องน้ำ ผิดปกติ เลยเอาข้อมูลที่ถูกมาใส่แทน
df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/บ้านปลายหาดคอนโด-วงอำมาตย์-1-นอน-ขายและให้เช่า-ขาย-9756220', 'bathroom'] = 1

# drop row ที่ bathroom อาจใส่ผิด หรือ เป็นการปล่อยเช่า
df_main.drop(index=df_main[(df_main['bathroom'] > 2) & (df_main['price'] < 1000000)].index, inplace=True)
# df_main[(df_main['bathroom'] > 2) & (df_main['price'] < 1000000)]

df_main['bathroom'] = pd.to_numeric(df_main['bathroom'], errors='coerce')


df_main['bathroom'].value_counts().sort_index(ascending=False)
# pd.set_option('display.max_colwidth', 400)


13        1
9        16
8        11
7        21
6       117
5       533
4      1461
3      4379
2     17700
1     54128
Name: bathroom, dtype: int64

In [5]:
# เจอ price ที่ใส่ผิด
df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/braf0828-q-langsuan-ทำเลทองหลังสวน-ใกล้ชิดลม-บนพื้นที่-freehold-ขาย-9948629', 'price'] = 25300000
df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/ขาย-ริชพาร์ค-บางซ่อน-ถูกที่สุด-2-29-ล้าน-28-96-ตร-ม-แต่งสวย-ติด-mrt-บางซ่อน-คุณบี-064-146-6445-s5575-ขาย-8054628', 'price'] = 29000000

# drop row ที่ price < 500k เพราะ มีราคาเช่า ปนเข้ามาด้วยเช่น price 20k, 200k แต่มีสิทธิ์ drop condo ราคาถูกๆไป
df_main.drop(index = df_main[(df_main['price'] <= 500000)].index, inplace=True)

df_main['price'].describe().apply(lambda x: format(x,'.2f'))

count          77782.00
mean        11375699.92
std        109165679.81
min           510000.00
25%          3150000.00
50%          5600000.00
75%         11400000.00
max      29900000000.00
Name: price, dtype: object

In [6]:
df_main['district'].value_counts().sort_values(ascending=False)

วัฒนา                14778
คลองเตย               9760
ห้วยขวาง              7786
จตุจักร               5294
ราชเทวี               3616
ปทุมวัน               3477
คลองสาน               3247
พระโขนง               3001
บางรัก                2725
สาทร                  2479
บางนา                 2081
ยานนาวา               1997
ดินแดง                1970
พญาไท                 1889
สวนหลวง               1826
บางซื่อ               1588
ธนบุรี                1581
บางกะปิ               1574
บางคอแหลม              828
ภาษีเจริญ              681
บางเขน                 647
บางพลัด                639
จอมทอง                 576
ประเวศ                 510
บึงกุ่ม                426
วังทองหลาง             398
ลาดพร้าว               396
ลาดกระบัง              380
คันนายาว               252
บางกอกใหญ่             240
ราษฎร์บูรณะ            228
บางกอกน้อย             218
บางแค                  202
บางขุนเทียน             96
หลักสี่                 93
มีนบุรี                 71
สายไหม                  52
ต

In [7]:
df_main['city'].value_counts().sort_values(ascending=False)

คลองตันเหนือ    8167
ห้วยขวาง        4472
คลองตัน         4248
พระโขนง         3866
คลองเตยเหนือ    3733
                ... 
หนองแขม            1
คูหาสวรรค์         1
บางพรม             1
กระทุ่มราย         1
ทับยาว             1
Name: city, Length: 131, dtype: int64

In [8]:

df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/ขายด่วน-คอนโด-กลางกรุงรีสอร์ท-รัชดา-7-รถไฟฟ้าห้วยขวาง-62-ตรม-ชั้น-8-ขาย-8976391', 'size'] = 62
df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/yog-peis007supalai-riva-grand-76sqm-ขาย-8850333', 'size'] = 76
df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/cd-620143-ขายwhizdom-essence-3-ห้องนอน-2-ห้องน้ำ-ขนาด-100-4ตรม-ติด-bts-ปุณวิถี-ขาย-7418353', 'size'] = 100.4



def edit_size(row, size, price):
    if (row['size'] > size) & (row['price'] < price):
        if (len(str(row['size']).split('.')[0]) - 2 < 0):
            
            return np.nan 
        
        else:
            return row['size']/(10**(len(str(row['size']).split('.')[0]) - 2))
    else:
        return row['size']

# edit size โดยเอา size หาร 10 หรือ 100
df_main.loc[:,'size'] = df_main.apply(edit_size, args=(200, 5000000), axis=1)
df_main.loc[:,'size'] = df_main.apply(edit_size, args=(500, 10000000), axis=1)

# df_main['size'].describe()
# pd.set_option('display.max_colwidth', 400)
# size_df_main = df_main[(df_main['size'] > 200) & (df_main['price'] < 10000000)]

In [9]:
# replace with nan
df_main['furniture'].replace('ไม่มีข้อมูล', np.nan, inplace=True)

df_main['furniture'].value_counts().sort_values(ascending=False)

ตกแต่งครบ    35345
มีบางส่วน     5819
ไม่มี         1401
Name: furniture, dtype: int64

In [10]:
# replace with nan
df_main['finish_year'].replace('ไม่มีข้อมูล', np.nan, inplace=True)

# change to numeric
df_main['finish_year'] = pd.to_numeric(df_main['finish_year'], errors='coerce')

# df_main['finish_year'].value_counts().sort_index(ascending=True)
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77782 entries, 0 to 78395
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   link              77782 non-null  object 
 1   district          77782 non-null  object 
 2   city              77782 non-null  object 
 3   price             77782 non-null  int64  
 4   bedroom           76983 non-null  float64
 5   bathroom          77782 non-null  int64  
 6   size              77677 non-null  float64
 7   distance_station  61968 non-null  float64
 8   furniture         42565 non-null  object 
 9   finish_year       63429 non-null  float64
 10  floor             77781 non-null  object 
dtypes: float64(4), int64(2), object(5)
memory usage: 7.1+ MB


In [11]:
# replace with nan
df_main['floor'].replace('ไม่มีข้อมูล', np.nan, inplace=True)

# change to numeric
df_main['floor'] = pd.to_numeric(df_main['floor'], errors='coerce')

df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/tta579-ขายถูกมากกกขาดทุน-800-000-30-ตร-ม-1-นอน-1-น้ำ-097-454-2624-ขาย-9993964','floor'] = np.nan
df_main.loc[df_main['link'] == 'https://www.ddproperty.com/property/คอนโด-fynn-aree-1-นอน-ชั้นสูง-ใกล้-bts-อารีย์-id-591760-ขาย-9692433','floor'] = 7
df_main.loc[(df_main['floor'] >= 90),'floor'] = np.nan



df_main['floor'].value_counts().sort_index(ascending=True)

1.0      602
2.0     2164
3.0     2230
4.0     2809
5.0     2539
        ... 
68.0       2
70.0       6
73.0       8
74.0      13
77.0      16
Name: floor, Length: 71, dtype: int64

In [12]:
df_main['distance_station'] = np.where(df_main['distance_station'].isnull(), 'ไม่ใกล้รถไฟฟ้า', 'ใกล้รถไฟฟ้า')

In [13]:
df_main.head()

,link,district,city,price,bedroom,bathroom,size,distance_station,furniture,finish_year,floor
0,https://www.ddproperty.com/property/ll-ls659-ข...,สวนหลวง,สวนหลวง,1870000,1.0,1,23.0,ใกล้รถไฟฟ้า,ไม่มี,2018.0,8.0
1,https://www.ddproperty.com/property/ys1583-ขาย...,จตุจักร,จันทรเกษม,1990000,1.0,1,30.0,ไม่ใกล้รถไฟฟ้า,NaN,2013.0,NaN
2,https://www.ddproperty.com/property/noble-reco...,วัฒนา,คลองเตยเหนือ,12000000,2.0,2,68.0,ใกล้รถไฟฟ้า,ตกแต่งครบ,2018.0,NaN
4,https://www.ddproperty.com/property/ขายด่วนค่า...,ลาดพร้าว,ลาดพร้าว,1580000,1.0,1,35.0,ไม่ใกล้รถไฟฟ้า,NaN,2013.0,6.0
5,https://www.ddproperty.com/property/ขายคอนโดรี...,พระโขนง,บางจาก,1250000,1.0,1,31.0,ใกล้รถไฟฟ้า,ตกแต่งครบ,NaN,7.0


In [14]:
# ปรับ index ให้เรียงตั้งแต่ 0 ใหม่ หลังการ drop บาง row
df_main.reset_index(drop=True, inplace=True)

In [15]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77782 entries, 0 to 77781
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   link              77782 non-null  object 
 1   district          77782 non-null  object 
 2   city              77782 non-null  object 
 3   price             77782 non-null  int64  
 4   bedroom           76983 non-null  float64
 5   bathroom          77782 non-null  int64  
 6   size              77677 non-null  float64
 7   distance_station  77782 non-null  object 
 8   furniture         42565 non-null  object 
 9   finish_year       63429 non-null  float64
 10  floor             45085 non-null  float64
dtypes: float64(4), int64(2), object(5)
memory usage: 6.5+ MB


# Split Data to Train set 70%, Validation set 15%, Test set 15%

In [16]:
from sklearn.model_selection import train_test_split

df, df_test = train_test_split(df_main, test_size=0.3, random_state=18)

df_test, df_valid = train_test_split(df_test, train_size=0.5, random_state=18)

# df, df_test = train_test_split(df_main, test_size=0.6, random_state=0)

# df_test, df_valid = train_test_split(df_test, train_size=0.5, random_state=0)

df.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_valid.reset_index(drop=True, inplace=True)

print(df.shape)
print(df_test.shape)
print(df_valid.shape)

(54447, 11)
(11667, 11)
(11668, 11)


# Training Set

In [17]:
# Show missing ratio
miss_no = df.isnull().sum()
miss_percent = round(df.isnull().sum()/df.shape[0]*100, 2)
table_miss = pd.concat([miss_no, miss_percent], axis=1)
table_miss.rename(columns={0:'missing_number', 1:'missing_percent'}, inplace=True)
table_miss[table_miss['missing_number'] != 0]




,missing_number,missing_percent
bedroom,540,0.99
size,73,0.13
furniture,24728,45.42
finish_year,10067,18.49
floor,22893,42.05


In [18]:
print('percent of row that has nan is ',(len(df[df.isnull().any(axis=1)])/len(df))*100)

percent of row that has nan is  71.28032765808952


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54447 entries, 0 to 54446
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   link              54447 non-null  object 
 1   district          54447 non-null  object 
 2   city              54447 non-null  object 
 3   price             54447 non-null  int64  
 4   bedroom           53907 non-null  float64
 5   bathroom          54447 non-null  int64  
 6   size              54374 non-null  float64
 7   distance_station  54447 non-null  object 
 8   furniture         29719 non-null  object 
 9   finish_year       44380 non-null  float64
 10  floor             31554 non-null  float64
dtypes: float64(4), int64(2), object(5)
memory usage: 4.6+ MB


# Cramer

In [20]:
# Create dummy variable for missing value
import numpy as np

df1 =df.copy()

for i in [ i for i in df.columns if df[i].isnull().any()]:
  df1[i+'_m'] = np.where(df1[i].isnull(),True,False)

df1.head(1)

,link,district,city,price,bedroom,bathroom,size,distance_station,furniture,finish_year,floor,bedroom_m,size_m,furniture_m,finish_year_m,floor_m
0,https://www.ddproperty.com/property/the-parkla...,ธนบุรี,บุคคโล,4700000,2.0,1,50.0,ใกล้รถไฟฟ้า,NaN,2010.0,NaN,False,False,True,False,True


In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54447 entries, 0 to 54446
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   link              54447 non-null  object 
 1   district          54447 non-null  object 
 2   city              54447 non-null  object 
 3   price             54447 non-null  int64  
 4   bedroom           53907 non-null  float64
 5   bathroom          54447 non-null  int64  
 6   size              54374 non-null  float64
 7   distance_station  54447 non-null  object 
 8   furniture         29719 non-null  object 
 9   finish_year       44380 non-null  float64
 10  floor             31554 non-null  float64
 11  bedroom_m         54447 non-null  bool   
 12  size_m            54447 non-null  bool   
 13  furniture_m       54447 non-null  bool   
 14  finish_year_m     54447 non-null  bool   
 15  floor_m           54447 non-null  bool   
dtypes: bool(5), float64(4), int64(2), object

In [22]:
print('Unique values')
for i in df1.columns.to_list():
  tmp = df1[i].nunique()
  print('{:<8}:{}'.format(i, tmp))

Unique values
link    :54447
district:50
city    :129
price   :5125
bedroom :8
bathroom:9
size    :519
distance_station:2
furniture:3
finish_year:39
floor   :71
bedroom_m:2
size_m  :2
furniture_m:2
finish_year_m:2
floor_m :2


In [23]:
# use qcut and convert to category type  
# bedroom 
# bathroom
# size
# finish_year
# floor
df2 = df1.copy()

df2['bedroom'] =pd.qcut(df2['bedroom'], q=3, duplicates='drop').astype('category')
df2['bathroom'] =pd.qcut(df2['bathroom'], q=4, duplicates='drop').astype('category')
df2['size'] =pd.qcut(df2['size'], q=10, duplicates='drop').astype('category')
df2['finish_year'] =pd.qcut(df2['finish_year'], q=10, duplicates='drop').astype('category')
df2['floor'] =pd.qcut(df2['floor'], q=10, duplicates='drop').astype('category')

# drop column y
df2.drop('price', axis=1, inplace=True)
df2.drop('link', axis=1, inplace=True)


# Convert 6 boolean columns that we create prevousely to category columns
df_category = df2.apply(lambda x: x.astype("category") if ((x.dtype == "bool") | (x.dtype == "O"))  else x)

df_category.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54447 entries, 0 to 54446
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   district          54447 non-null  category
 1   city              54447 non-null  category
 2   bedroom           53907 non-null  category
 3   bathroom          54447 non-null  category
 4   size              54374 non-null  category
 5   distance_station  54447 non-null  category
 6   furniture         29719 non-null  category
 7   finish_year       44380 non-null  category
 8   floor             31554 non-null  category
 9   bedroom_m         54447 non-null  category
 10  size_m            54447 non-null  category
 11  furniture_m       54447 non-null  category
 12  finish_year_m     54447 non-null  category
 13  floor_m           54447 non-null  category
dtypes: category(14)
memory usage: 807.7 KB


In [24]:
df2['bedroom'].value_counts()
df2['bathroom'].value_counts()
df2['size'].value_counts()
df2['finish_year'].value_counts()
df2['floor'].value_counts()



(3.0, 5.0]      3776
(5.0, 7.0]      3721
(0.999, 3.0]    3473
(14.0, 18.0]    3403
(28.0, 77.0]    3119
(8.0, 11.0]     3078
(18.0, 22.0]    3052
(22.0, 28.0]    2992
(11.0, 14.0]    2803
(7.0, 8.0]      2137
Name: floor, dtype: int64

In [25]:
# Cramer >.5 (high), 0.3-0.5 (mediam), 0.1-0.3(low) <.1(no association)

# มีแค่ distance_station อย่างเดียวที่มี association กับ column อื่นๆ(ไม่ใช่ MCAR)  นอกนั้นไม่มี association >>> ต้อง Imputation
import association_metrics as am

# Initialize a Cramer V object
cramersv = am.CramersV(df_category)

# Generate the pairwise matrix filled with Cramer V
# cramersv.fit()
cramersv.fit().iloc[0:9, 9:]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/association_metrics/categorical.py:78: RuntimeWarning: invalid value encountered in double_scalars
  v = sqrt((chi2/n)/mindim)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/association_metrics/categorical.py:78: RuntimeWarning: invalid value encountered in double_scalars
  v = sqrt((chi2/n)/mindim)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/association_metrics/categorical.py:78: RuntimeWarning: invalid value encountered in double_scalars
  v = sqrt((chi2/n)/mindim)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/association_metrics/categorical.py:78: RuntimeWarning: invalid value encountered in double_scalars
  v = sqrt((chi2/n)/mindim)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/association_metrics/categorical.py:78: RuntimeWarning: invalid value encountered in double_scalars
  v

,bedroom_m,size_m,furniture_m,finish_year_m,floor_m
district,0.067470,0.098415,0.083603,0.117359,0.254163
city,0.082733,0.173389,0.112024,0.161764,0.291321
bedroom,NaN,0.003295,0.006280,0.111943,0.118987
bathroom,0.027626,0.003387,0.005587,0.108134,0.129210
size,0.076839,NaN,0.030071,0.144112,0.250508
distance_station,0.018837,0.025072,0.010520,0.010126,0.105468
furniture,0.029110,0.017332,NaN,0.078914,0.086913
finish_year,0.035023,0.012022,0.077639,NaN,0.105731
floor,0.017265,0.023017,0.018884,0.047621,NaN


# Ordinal Encoding

In [26]:
# Imputation with Ordinal encoding + IterativeImputation
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

df_model = df.drop('link', axis=1).copy()

from sklearn.preprocessing import LabelEncoder


furniture = ['ไม่มี', 'มีบางส่วน', 'ตกแต่งครบ']
station = ['ไม่ใกล้รถไฟฟ้า', 'ใกล้รถไฟฟ้า']

# Ordinal encoding
def ord_na(x, colname):
    from sklearn.preprocessing import OrdinalEncoder

    if colname == 'furniture':
        le = OrdinalEncoder(handle_unknown='use_encoded_value', categories=[furniture],
                                    unknown_value=-1, dtype='int64')
    elif colname == 'distance_station':
        le = OrdinalEncoder(handle_unknown='use_encoded_value', categories=[station],
                                    unknown_value=-1, dtype='int64')
    else:
        le = OrdinalEncoder(handle_unknown='use_encoded_value',
                                    unknown_value=-1, dtype='int64')    

    ### fit with the desired col, col in position 0 for this example
    fit_by = pd.Series([i for i in x[colname].unique() if type(i) == str])
    le.fit(np.array(fit_by).reshape(-1, 1))

    ### Set transformed col leaving np.NaN as they are
    x[colname] = x[colname].apply(lambda x: le.transform(np.array([x]).reshape(-1,1))[0][0] if type(x) == str else x)
    return x, le

df_model, le_district = ord_na(df_model, 'district')
df_model, le_city = ord_na(df_model, 'city')
df_model, le_distance_station = ord_na(df_model, 'distance_station')
df_model, le_furniture = ord_na(df_model, 'furniture')


# # Imputation missing data
def imputer_value(data):
        
    imp = IterativeImputer() 
    imp.fit(data.drop('price',axis=1)) # create model with df no col Y
    tmp = imp.transform(data.drop('price',axis=1)) # impute nan
    tmp = pd.DataFrame(tmp, columns=data.drop('price',axis=1).columns) # create df from all col X
    df_temp = pd.concat([tmp, data['price']],axis=1)

    # Round value to interger after imputing except col size and price
    for col in df_temp.columns:
        if col not in ['size', 'price']:
            df_temp[col] = df_temp[col].apply(round)


    return df_temp, imp
# No impute
#impute_model, imp = imputer_value(df_model)
#df_model['floor'] = impute_model['floor']

df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54447 entries, 0 to 54446
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   district          54447 non-null  int64  
 1   city              54447 non-null  int64  
 2   price             54447 non-null  int64  
 3   bedroom           53907 non-null  float64
 4   bathroom          54447 non-null  int64  
 5   size              54374 non-null  float64
 6   distance_station  54447 non-null  int64  
 7   furniture         29719 non-null  float64
 8   finish_year       44380 non-null  float64
 9   floor             31554 non-null  float64
dtypes: float64(5), int64(5)
memory usage: 4.2 MB


# Export Ordinal encoding model and Iterative model for encoding real data

In [280]:
import joblib

def export_object(obj, filename):
    joblib.dump(obj , filename)

export_object(le_district, 'le_district.txt')
export_object(le_city, 'le_city.txt')
export_object(le_distance_station, 'le_distance_station.txt')
export_object(le_furniture, 'le_furniture.txt')

# export_object(imp, 'IterativeImputer.txt')







# Detect + Remove Outlier

In [27]:
from sklearn.ensemble import IsolationForest

# No imputation. replace NaN with -99 before Detecing Outlier
df_model.replace(np.nan, -99, inplace=True)

iforest = IsolationForest(n_estimators=1000, contamination=0.05)
# iforest = IsolationForest(n_estimators=200)
iforest.fit(df_model)

cluster = iforest.predict(df_model)

df_outlier = df_model.copy()
df_outlier['outlier'] = cluster

# Outlier 5%
print(np.mean(df_outlier['outlier'] == -1))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


0.050011938215145005


In [28]:
# Delete outlier row
df_model = df_outlier[df_outlier['outlier'] != -1].drop('outlier', axis=1)

df_model.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 51724 entries, 0 to 54446
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   district          51724 non-null  int64  
 1   city              51724 non-null  int64  
 2   price             51724 non-null  int64  
 3   bedroom           51724 non-null  float64
 4   bathroom          51724 non-null  int64  
 5   size              51724 non-null  float64
 6   distance_station  51724 non-null  int64  
 7   furniture         51724 non-null  float64
 8   finish_year       51724 non-null  float64
 9   floor             51724 non-null  float64
dtypes: float64(5), int64(5)
memory usage: 4.3 MB


In [29]:
# Replace -99 to NaN
df_model.replace(-99,np.nan, inplace=True)
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51724 entries, 0 to 54446
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   district          51724 non-null  int64  
 1   city              51724 non-null  int64  
 2   price             51724 non-null  int64  
 3   bedroom           51287 non-null  float64
 4   bathroom          51724 non-null  int64  
 5   size              51667 non-null  float64
 6   distance_station  51724 non-null  int64  
 7   furniture         28342 non-null  float64
 8   finish_year       43033 non-null  float64
 9   floor             30424 non-null  float64
dtypes: float64(5), int64(5)
memory usage: 4.3 MB


# XG Boost model

In [36]:
# xg boost model with test set
import xgboost as xgb
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def xg_model_fit(df):


        X = df[[col for col in df.columns if col!="price"]]
        y = df["price"]

        # # Baseline model with default parameter
        # xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')

        ## Random hyper parameter
        #xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.5, learning_rate = 0.01,
        #         max_depth = 100, alpha = 10, n_estimators = 200)

        # After tuning hyper parameter
        xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.8, learning_rate = 0.05,
                max_depth = 40, alpha = 5, n_estimators = 100)
        


        xg_reg.fit(X,y)

        return xg_reg
  
def xg_model_predict(model, df):
        X = df[[col for col in df.columns if col!="price"]]
        y = df["price"]
        preds = model.predict(X)


        # return MAPE, prediction value, model
        return mean_absolute_percentage_error(preds, y), preds

In [37]:
# Imputation with xgboost
model_predict = xg_model_fit(df_model)
tr_MAPE, _ = xg_model_predict(model_predict, df_model)


print('xgboost train MAPE:',tr_MAPE)


# **************************************************
# Baseline model for xgboost with default parameter (No delete outlier)
# xgboost train MAPE: 0.3064

# Baseline model for xgboost with randomes parameter (No delete outlier)
# xgboost train MAPE: 0.23256658567715696

# Interger no impute + tuning
# xgboost train MAPE: 0.049271693755489246

# grid search with xgboost
# Fitting 5 folds for each of 320 candidates, totalling 1600 fits
# Best parameters: {'alpha': 5, 'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 40, 'n_estimators': 100}
# Lowest MAPE:  -0.21455428938557186  



xgboost train MAPE: 0.04927212552642515


# Validation Set

In [38]:

df_val = df_valid.copy()
df_val = df_val.drop('link', axis =1)


# Ordinal encoding each categorical column
### Set transformed col leaving np.NaN as they are
df_val['district'] = df_val['district'].apply(lambda x: le_district.transform(np.array([x]).reshape(-1,1))[0][0] if type(x) == str else x)
df_val['city'] = df_val['city'].apply(lambda x: le_city.transform(np.array([x]).reshape(-1,1))[0][0] if type(x) == str else x)
df_val['distance_station'] = df_val['distance_station'].apply(lambda x: le_distance_station.transform(np.array([x]).reshape(-1,1))[0][0] if type(x) == str else x)
df_val['furniture'] = df_val['furniture'].apply(lambda x: le_furniture.transform(np.array([x]).reshape(-1,1))[0][0] if type(x) == str else x)

# Imputing NaN
# df_val, _ = imputer_value(df_val)

# Change to float before predict with xgboost
# for col in df_val.columns:
#     df_val[col] = df_val[col].astype('float64')

df_val.info()
df_val.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11668 entries, 0 to 11667
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   district          11668 non-null  int64  
 1   city              11668 non-null  int64  
 2   price             11668 non-null  int64  
 3   bedroom           11552 non-null  float64
 4   bathroom          11668 non-null  int64  
 5   size              11650 non-null  float64
 6   distance_station  11668 non-null  int64  
 7   furniture         6400 non-null   float64
 8   finish_year       9509 non-null   float64
 9   floor             6717 non-null   float64
dtypes: float64(5), int64(5)
memory usage: 911.7 KB


,district,city,price,bedroom,bathroom,size,distance_station,furniture,finish_year,floor
0,40,80,4326000,2.0,1,49.0,1,1.0,NaN,6.0
1,40,80,7756000,1.0,1,53.0,1,NaN,2011.0,NaN
2,33,84,1900000,1.0,1,29.0,0,2.0,2018.0,2.0
3,4,91,7500000,2.0,2,79.0,1,NaN,2009.0,NaN
4,17,71,7900000,2.0,2,77.0,1,2.0,NaN,27.0


In [39]:
# Predict validation set with xgboost

val_MAPE, _ = xg_model_predict(model_predict, df_val)


print('xgboost validation MAPE:',val_MAPE)


# Interger no impute
# xgboost validation MAPE: 1.0698844593037513

# Interger no impute + tuning
# xgboost validation MAPE: 0.19086381060805485

xgboost validation MAPE: 0.1926447279724479


In [132]:
# Create GridSearchCV to find best parameter of xgboost
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Create grid search cv
def xg_model_grid(df):
    X, y = df.drop('price', axis=1), df['price']

    # Trial paramter
    params = {
            'max_depth': [35, 40, 45, 50],
            'learning_rate': [0.05, 0.1, 0.15, 0.2],
            'n_estimators': [80, 100, 150, 300],
            'colsample_bytree': [0.7, 0.8],
            'alpha': [4, 5, 6]}
    xgbr = xgb.XGBRegressor(seed = 20)
    clf = GridSearchCV(estimator=xgbr, 
                    param_grid=params,
                    cv=5,
                #     scoring='neg_mean_squared_error', 
                    scoring='neg_mean_absolute_percentage_error',
                    verbose=1)
    clf.fit(X, y)
    print("Best parameters:", clf.best_params_)
    print("Lowest MAPE: ", clf.best_score_)





# Try GridSearchCV to find best parameter of xgboost
print('grid search with xgboost')
xg_model_grid(df_val)



grid search with xgboost
Fitting 5 folds for each of 384 candidates, totalling 1920 fits


KeyboardInterrupt: 

# Test Set

In [40]:

df_t = df_test.copy()
df_t = df_t.drop('link', axis =1)



# Ordinal encoding each categorical column
### Set transformed col leaving np.NaN as they are
df_t['district'] = df_t['district'].apply(lambda x: le_district.transform(np.array([x]).reshape(-1,1))[0][0] if type(x) == str else x)
df_t['city'] = df_t['city'].apply(lambda x: le_city.transform(np.array([x]).reshape(-1,1))[0][0] if type(x) == str else x)
df_t['distance_station'] = df_t['distance_station'].apply(lambda x: le_distance_station.transform(np.array([x]).reshape(-1,1))[0][0] if type(x) == str else x)
df_t['furniture'] = df_t['furniture'].apply(lambda x: le_furniture.transform(np.array([x]).reshape(-1,1))[0][0] if type(x) == str else x)


# # Imputing NaN
# df_t, _ = imputer_value(df_t)

# # Change to float before predict with xgboost
# for col in df_t.columns:
#     df_t[col] = df_t[col].astype('float64')

df_t.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11667 entries, 0 to 11666
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   district          11667 non-null  int64  
 1   city              11667 non-null  int64  
 2   price             11667 non-null  int64  
 3   bedroom           11524 non-null  float64
 4   bathroom          11667 non-null  int64  
 5   size              11653 non-null  float64
 6   distance_station  11667 non-null  int64  
 7   furniture         6446 non-null   float64
 8   finish_year       9540 non-null   float64
 9   floor             6814 non-null   float64
dtypes: float64(5), int64(5)
memory usage: 911.6 KB


In [41]:
# Predict validation set with xgboost

test_MAPE, _ = xg_model_predict(model_predict, df_t)


print('xgboost test MAPE:',test_MAPE)

# Integer no impute
# xgboost test MAPE: 0.20596826479811414

# Integer no impute + tuning
# xgboost test MAPE: 0.194314731748161

xgboost test MAPE: 0.24361850563904186


# Try to predict 5 record with model

In [42]:
df_t

,district,city,price,bedroom,bathroom,size,distance_station,furniture,finish_year,floor
0,40,5,4500000,1.0,1,36.0,0,NaN,2015.0,NaN
1,4,19,650000,1.0,1,32.0,1,NaN,2012.0,NaN
2,49,45,5390000,NaN,1,30.0,1,NaN,2018.0,NaN
3,19,56,2700000,1.0,1,37.0,0,NaN,2014.0,3.0
4,34,21,23500000,3.0,3,250.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
11662,12,111,15500000,3.0,3,117.0,1,2.0,NaN,14.0
11663,19,56,5700000,2.0,1,40.0,1,2.0,2018.0,30.0
11664,32,66,2800000,1.0,1,44.0,1,NaN,NaN,8.0
11665,0,65,2900000,1.0,1,32.0,0,NaN,NaN,NaN


In [43]:
# Try predict 1 record
true_y = df_t.iloc[0:5,2]
# df_t.iloc[0:1,5] = 0

pred_y = model_predict.predict(df_t.iloc[0:5,].drop('price', axis=1))

print('true_y:',true_y.to_list())
print('pred_y:',pred_y.tolist())

true_y: [4500000, 650000, 5390000, 2700000, 23500000]
pred_y: [4567214.5, 3524310.5, 4800471.0, 2499898.75, 16753671.0]


In [47]:
df_t.loc[df_t['city'] == 122,:]
df_test.iloc[21,]

link                https://www.ddproperty.com/property/ขายคอนโดรา...
district                                                     ห้วยขวาง
city                                                         ห้วยขวาง
price                                                         3400000
bedroom                                                           1.0
bathroom                                                            1
size                                                             40.0
distance_station                                          ใกล้รถไฟฟ้า
furniture                                                   ตกแต่งครบ
finish_year                                                    2009.0
floor                                                             NaN
Name: 21, dtype: object

In [45]:
df_t.iloc[0:5,]

,district,city,price,bedroom,bathroom,size,distance_station,furniture,finish_year,floor
0,40,5,4500000,1.0,1,36.0,0,NaN,2015.0,NaN
1,4,19,650000,1.0,1,32.0,1,NaN,2012.0,NaN
2,49,45,5390000,NaN,1,30.0,1,NaN,2018.0,NaN
3,19,56,2700000,1.0,1,37.0,0,NaN,2014.0,3.0
4,34,21,23500000,3.0,3,250.0,0,NaN,NaN,NaN


In [46]:
df_test.head(5)

,link,district,city,price,bedroom,bathroom,size,distance_station,furniture,finish_year,floor
0,https://www.ddproperty.com/property/pet-friend...,วัฒนา,คลองตันเหนือ,4500000,1.0,1,36.0,ไม่ใกล้รถไฟฟ้า,NaN,2015.0,NaN
1,https://www.ddproperty.com/property/ขาย-คอนโด-...,จตุจักร,จอมพล,650000,1.0,1,32.0,ใกล้รถไฟฟ้า,NaN,2012.0,NaN
2,https://www.ddproperty.com/property/condo-for-...,ห้วยขวาง,บางกะปิ,5390000,NaN,1,30.0,ใกล้รถไฟฟ้า,NaN,2018.0,NaN
3,https://www.ddproperty.com/property/bc-01595-ข...,บางนา,บางนา,2700000,1.0,1,37.0,ไม่ใกล้รถไฟฟ้า,NaN,2014.0,3.0
4,https://www.ddproperty.com/property/the-lanai-...,ยานนาวา,ช่องนนทรี,23500000,3.0,3,250.0,ไม่ใกล้รถไฟฟ้า,NaN,NaN,NaN


# Try predict mean of type room (every column are the same, except price column)

In [48]:
df_try = df_t.groupby(['district', 'city', 'bedroom', 'bathroom', 'size', 'distance_station', 'furniture', 'finish_year', 'floor']).agg({'price':['count', 'mean']}).reset_index().sort_values(ascending = False, by = ('price', 'count'))
df_try.head(4)

district city bedroom bathroom  size distance_station furniture  \
                                                                       
3043       49  122     1.0        1  39.0                1       2.0   
2880       49  108     1.0        1  28.0                0       1.0   
3090       49  122     2.0        1  49.0                1       2.0   
3080       49  122     1.0        1  48.0                1       2.0   

     finish_year floor price                
                       count          mean  
3043      2014.0   2.0     7  3.170000e+06  
2880      2015.0   8.0     6  1.441667e+06  
3090      2014.0  22.0     6  6.000000e+06  
3080      2015.0  16.0     5  6.370000e+06

In [49]:
# Try predict mean of type room
true_y = df_try.iloc[0:4,10]
# df_t.iloc[0:1,5] = 0

pred_y = model_predict.predict(df_try.iloc[0:4,0:9])

print('true_y:',true_y.to_list())
print('pred_y:',pred_y.tolist())

true_y: [3170000.0, 1441666.6666666667, 6000000.0, 6370000.0]
pred_y: [3150143.25, 1451586.875, 5920527.0, 6314662.0]


# Export xgboost model for using in production

In [298]:
import joblib

def export_object(obj, filename):
    joblib.dump(obj , filename)

export_object(model_predict, 'model_predict.txt')


# def import_object(filename):
#     object_Json = joblib.load(filename)

#     return object_Json

# le_province = import_object('le_province.txt')
# le_province.transform(np.array(['โอเค']).reshape(-1,1))[0][0]






# Import model from file

In [299]:
import joblib
def import_object(filename):
    object_Json = joblib.load(filename)

    return object_Json

production_model = import_object('model_predict.txt')


# Try predict 1 record
true_y = df_model.iloc[0:1,2]
pred_y = production_model.predict(df_model.iloc[0:1,].drop('price', axis=1))

print('true_y:',true_y.to_list())
print('pred_y:',pred_y.tolist())


true_y: [4700000]
pred_y: [4373904.0]


In [300]:
df_model.head(1)

,district,city,price,bedroom,bathroom,size,distance_station,furniture,finish_year,floor
0,12,73,4700000,2.0,1,50.0,1,NaN,2010.0,NaN


In [301]:
df.head(1)

,link,district,city,price,bedroom,bathroom,size,distance_station,furniture,finish_year,floor
0,https://www.ddproperty.com/property/the-parkla...,ธนบุรี,บุคคโล,4700000,2.0,1,50.0,ใกล้รถไฟฟ้า,NaN,2010.0,NaN
